In [72]:
from bs4 import BeautifulSoup
import requests
from spacy.lang.en import English
import pandas as pd
import numpy as np
import math
nlp = English()

In [73]:
BASE_URL = [
'https://thenewstack.io/can-we-teach-an-ai-to-play-dungeons-and-dragons/',
'https://medium.com/analytics-vidhya/a-discord-bot-for-dungeons-and-dragons-part-1-f967561ad73e',
'https://expmag.com/2020/03/i-slayed-dragons-with-an-ai-powered-dungeon-master/',
'https://developer.mantiumai.com/docs/securing-a-dungeons-and-dragons-assistant',
'http://www.viviancook.uk/Writings/Papers/System86.htm'
]

In [74]:
output = ''
base_text = []
for url in BASE_URL:
  res = requests.get(url)
  html_page = res.content
  soup = BeautifulSoup(html_page, 'html.parser')
  text = soup.find_all(text=True)
  for data in soup.find_all("p"):
    temp = str(data.get_text())
    base_text.append(temp)
base_text = [element.split(".") for element in base_text]
base_text = [item for sublist in base_text for item in sublist]
base_text = [element.split(" ") for element in base_text]

In [75]:
BoW = pd.DataFrame()
for list in base_text:
  count = pd.Series(list).value_counts()
  BoW = pd.concat([BoW,count], axis = 1)


BoW.columns = range(BoW.shape[1])
BoW = BoW.T

In [76]:
IDF = BoW.isna().sum()
n = len(BoW.index)
BoW = BoW.fillna(0)

In [77]:
TF = BoW.div(BoW.sum(axis=1), axis=0)
display(TF)

,a,by,the,AI,There,"magazine,",generated,story,shared,Martin,...,London,leave?,half,ten,platform,Platform,arrive?,arrives,eleven,Goodbye
0,0.096774,0.064516,0.064516,0.064516,0.032258,0.032258,0.032258,0.032258,0.032258,0.032258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
774,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
775,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
IDF = IDF.apply(lambda x: math.log10((n)/(n-x)))
display(IDF)

a           0.763316
by          1.475448
the         0.570275
AI          1.811240
There       2.589391
              ...   
Platform    2.890421
arrive?     2.890421
arrives     2.890421
eleven      2.890421
Goodbye     2.589391
Length: 2790, dtype: float64

In [79]:
TF_IDF = TF*IDF

Essa celula demora mas termina de rodar

In [81]:
cos_sim = []
for i in range(n):
  cos_sim.append([])
  for j in range(n):
    a = TF_IDF.loc[i,:].to_numpy()
    b = TF_IDF.loc[j,:].to_numpy()
    cos_sim[i].append(np.dot(a,b)/(np.linalg.norm(a) * np.linalg.norm(b)))

In [82]:
cos_sim = pd.DataFrame(cos_sim).dropna()

In [83]:
display(cos_sim)

,0,1,2,3,4,5,6,7,8,9,...,767,768,769,770,771,772,773,774,775,776
0,1.000000,0.000000,0.046716,0.0,0.061689,0.039257,0.0,0.013386,0.040020,0.056914,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.000000,0.0,0.000000,0.011480,0.0,0.000000,0.011535,0.016643,...,0.077268,0.039631,1.000000,0.071895,1.000000,0.079334,1.000000,0.078979,1.000000,0.107751
2,0.046716,0.000000,1.000000,0.0,0.022244,0.118483,0.0,0.000000,0.043309,0.158310,...,0.034229,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.061689,0.000000,0.022244,0.0,1.000000,0.009605,0.0,0.008124,0.265245,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,0.000000,0.079334,0.000000,0.0,0.000000,0.000911,0.0,0.000000,0.000915,0.001320,...,0.022823,0.156832,0.079334,0.029001,0.079334,1.000000,0.079334,0.714841,0.079334,0.008548
773,0.000000,1.000000,0.000000,0.0,0.000000,0.011480,0.0,0.000000,0.011535,0.016643,...,0.077268,0.039631,1.000000,0.071895,1.000000,0.079334,1.000000,0.078979,1.000000,0.107751
774,0.000000,0.078979,0.000000,0.0,0.000000,0.000907,0.0,0.000000,0.000911,0.001314,...,0.210795,0.015915,0.078979,0.291365,0.078979,0.714841,0.078979,1.000000,0.078979,0.008510
775,0.000000,1.000000,0.000000,0.0,0.000000,0.011480,0.0,0.000000,0.011535,0.016643,...,0.077268,0.039631,1.000000,0.071895,1.000000,0.079334,1.000000,0.078979,1.000000,0.107751
